In [1]:
"""
def break_mha_into_slices(input_mha_file):
    pass
    sitk_image = sitk.ReadImage(input_mha_file)
    print(sitk_image)
    ct_slices = sitk.GetArrayFromImage(sitk_image)
    print(np.shape(ct_slices))
    spacing = sitk_image.GetSpacing()
    seriesuid, _ = os.path.splitext(os.path.basename(input_mha_file))
    slices = []
    # if not os.path.exists(output_folder):
    #     os.makedirs(output_folder)

    for i, slice_data in enumerate(ct_slices):
        slice_image = sitk.GetImageFromArray(slice_data)
        slice_image.SetOrigin(sitk_image.GetOrigin())
        slice_image.SetMetaData("SeriesUID", seriesuid)

        x_coordinate = slice_image.GetOrigin()[0]
        y_coordinate = slice_image.GetOrigin()[1]

        slice_image.SetMetaData("ImagePositionPatient", f"{x_coordinate} {y_coordinate}")
        
        if isinstance(spacing, tuple) and len(spacing) >= 3:
            slice_thickness = spacing[2]
        else:
            slice_thickness = None
        slice_image.SetMetaData("SliceThickness", str(slice_thickness))
        pixel_spacing_str = " ".join(map(str, spacing))
        slice_image.SetMetaData("PixelSpacing", pixel_spacing_str)
        slices.append(slice_image)

    return slices
"""
        # slice_filename = os.path.join(output_folder, f"slice_{i:04d}.mha")
        # sitk.WriteImage(slice_image, slice_filename)

'\ndef break_mha_into_slices(input_mha_file):\n    pass\n    sitk_image = sitk.ReadImage(input_mha_file)\n    print(sitk_image)\n    ct_slices = sitk.GetArrayFromImage(sitk_image)\n    print(np.shape(ct_slices))\n    spacing = sitk_image.GetSpacing()\n    seriesuid, _ = os.path.splitext(os.path.basename(input_mha_file))\n    slices = []\n    # if not os.path.exists(output_folder):\n    #     os.makedirs(output_folder)\n\n    for i, slice_data in enumerate(ct_slices):\n        slice_image = sitk.GetImageFromArray(slice_data)\n        slice_image.SetOrigin(sitk_image.GetOrigin())\n        slice_image.SetMetaData("SeriesUID", seriesuid)\n\n        x_coordinate = slice_image.GetOrigin()[0]\n        y_coordinate = slice_image.GetOrigin()[1]\n\n        slice_image.SetMetaData("ImagePositionPatient", f"{x_coordinate} {y_coordinate}")\n        \n        if isinstance(spacing, tuple) and len(spacing) >= 3:\n            slice_thickness = spacing[2]\n        else:\n            slice_thickness = N

In [2]:
import os
from sybil import Serie, Sybil
import time
import csv
import traceback
import pandas as pd
import argparse
import numpy as np
from tqdm import tqdm

In [3]:
"""
This script is used to run inference on a set of .mha files and save the results to a csv file.
"""


def get_subfolder_paths(parent_folder):
    """Return a list of all subfolder paths in a parent folder.
    Parameters:
    parent_folder (str): The path to the parent folder.
    Returns:s
    List[str]: A list of paths for all subfolders in the parent folder.
    """
    subfolder_names = os.listdir(parent_folder)
    subfolder_paths = [os.path.join(parent_folder, name) for name in subfolder_names if os.path.isdir(os.path.join(parent_folder, name))]
    return subfolder_paths

def get_subfolder_paths_from_csv(csv_file_path,path_to_mha):
    """Return a list of all subfolder paths extracted from a CSV file.
    
    Parameters:
    csv_file_path (str): The path to the CSV file.
    
    Returns:
    List[str]: A list of paths for all subfolders extracted from the CSV file.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)
    series_instance_ids = df['SeriesInstanceUID']+".mha"
    subfolder_paths = [os.path.join(path_to_mha, str(series_instance_id)) for series_instance_id in series_instance_ids]
    return subfolder_paths

def get_mha_filepaths(folder_path):
    """Parse through a folder of .mha files of CT slices and return a list of all file paths.
    Parameters:
    folder_path (str): The path to the folder containing the .mha files.
    Returns:
    List[str]: A list of file paths for all .mha files in the folder.
    """
    mha_filepaths = []
    for root, _, files in os.walk(folder_path):
        for file in sorted(files):  # Sort the files list
            if file.endswith(".mha") or file.endswith(".dcm"):
                mha_filepaths.append(os.path.join(root, file))
    return mha_filepaths


# Initialize an empty dictionary
data_dict = {}

def collectscores(seriesuid, scores):
    global data_dict

    keys = ['SeriesInstanceUID', 'year1', 'year2', 'year3', 'year4', 'year5', 'year6']
    values = [
        os.path.basename(seriesuid),
        scores[0][0],
        scores[0][1],
        scores[0][2],
        scores[0][3],
        scores[0][4],
        scores[0][5]
    ]
    # Use the seriesuid as the key for this entry in the data_dict
    entry_key = os.path.basename(seriesuid)
    data_dict[entry_key] = dict(zip(keys, values))

    return data_dict

def save_data_as_csv(data_dict, output_filename):

    with open(output_filename, "w", newline='') as csvfile:
        pass

    with open(output_filename, "w", newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)

        writer.writeheader()

        for series_key, series_value in data_dict.items():
            writer.writerow(series_value)

# def log_error(error_message, subfolder):
#     with open(log_file, "a") as log:
#         log.write("Error in subfolder '{}': {}\n".format(subfolder, error_message))
#         traceback.print_exc(file=log)

headers = ["SeriesInstanceUID", "year1", "year2", "year3", "year4", "year5", "year6"]
output = r"/data/bodyct/experiments/kristina_sybil/files/mha3d_cancer_enriched_windowing64_allagain.csv"
csvfile = r"/data/bodyct/experiments/kristina_sybil/temp/temp_output_files_sybil/fennie/combinedsybilkiran_all.csv"
path_to_mha = r"/data/bodyct/experiments/0-dlcst-mha"

In [4]:
# Load a trained model
# path = r"/data/bodyct/experiments/kristina_sybil/temp/test_data/mha/indeterminate/1.2.840.113704.1.111.6252.1187022321.14.mha"
print("loading")
model = Sybil("sybil_ensemble")
# 1.2.840.113704.1.111.340.1158160809.11.mha
# "W:\experiments\kristina_sybil\temp\test_data\mha\indeterminate\1.2.840.113704.1.111.340.1158160809.11.mha"

loading
Loaded model from /home/kristinaavramova/.sybil/28a7cd44f5bcd3e6cc760b65c7e0d54d.ckpt
Loaded model from /home/kristinaavramova/.sybil/56ce1a7d241dc342982f5466c4a9d7ef.ckpt
Loaded model from /home/kristinaavramova/.sybil/624407ef8e3a2a009f9fa51f9846fe9a.ckpt
Loaded model from /home/kristinaavramova/.sybil/64a91b25f84141d32852e75a3aec7305.ckpt
Loaded model from /home/kristinaavramova/.sybil/65fd1f04cb4c5847d86a9ed8ba31ac1a.ckpt


In [5]:
scans = get_subfolder_paths_from_csv(csvfile,path_to_mha)[424:]
print(scans)
for scan in tqdm(scans):
    try:
        serie = Serie([scan], mha3d=True)
        scores = model.predict([serie])
        data_dict = collectscores(scan, scores[0])
        save_data_as_csv(data_dict, output)
        print(scores)
    except Exception as e:
        # log_error(str(e), subfolder)
        print(e)

# # end_time_model = time.time()
# # print(f"Time taken for my_function: {end_time_model - start_time_model} seconds")

['/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6164.1116343417.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6184.1133193556.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6188.1109691842.10.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6216.1133886506.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6220.1125590447.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6252.1187022321.14.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6260.1102521829.10.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.628.1102346195.10.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6288.1110382975.10.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6304.1130429073.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6324.1199807849.11.mha', '/data/bodyct/experiments/0-dlcst-mha/1.2.840.113704.1.111.6352.1119363877.1

  0%|          | 0/175 [00:00<?, ?it/s]

(309, 512, 512)
(309, 512, 512)
(309, 512, 512)
(309, 512, 512)
(309, 512, 512)
(309, 512, 512)


  1%|          | 1/175 [00:44<2:08:49, 44.42s/it]

Prediction(scores=[[0.0, 0.0015740740740740743, 0.0029513487265095673, 0.006847014817261502, 0.010381396771807386, 0.01728707626903897]])
(342, 512, 512)
(342, 512, 512)
(342, 512, 512)
(342, 512, 512)
(342, 512, 512)
(342, 512, 512)


...

Prediction(scores=[[0.0011701163019909324, 0.002554438672368111, 0.007834902850118133, 0.012796834998944757, 0.01922933035321872, 0.03295727072916815]])
(367, 512, 512)
(367, 512, 512)
(367, 512, 512)
(367, 512, 512)
(367, 512, 512)
(367, 512, 512)


 99%|█████████▉| 174/175 [2:24:48<00:53, 53.36s/it]

Prediction(scores=[[0.0, 0.0015740740740740743, 0.004032429807590649, 0.006847014817261502, 0.010381396771807386, 0.019111393468237536]])
(334, 512, 512)
(334, 512, 512)
(334, 512, 512)
(334, 512, 512)
(334, 512, 512)
(334, 512, 512)


100%|██████████| 175/175 [2:25:39<00:00, 49.94s/it]

Prediction(scores=[[0.05741631638052795, 0.11387587948994367, 0.15048689721346972, 0.17213596051472665, 0.19081123978149148, 0.24696671593223316]])
